#### Import Necessari

In [11]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import random
from sklearn.multiclass import OneVsOneClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import accuracy_score, precision_score, recall_score

#### Caricamento Dataset

In [12]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset19-20.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [13]:
X_train = trainSet.drop("risultato", axis=1)
y_train = trainSet["risultato"]

X_validation = validationSet.drop("risultato", axis=1)
y_validation = validationSet["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]


In [14]:
X_train_noName = X_train
X_train_noName = X_train_noName.drop("squadraa", axis=1)
X_train_noName = X_train_noName.drop("squadrab", axis=1)
X_train_noName = X_train_noName.astype(float)
y_train=y_train.astype(float)

X_validation_noName = X_validation
X_validation_noName = X_validation_noName.drop("squadraa", axis=1)
X_validation_noName = X_validation_noName.drop("squadrab", axis=1)
X_validation_noName = X_validation_noName.astype(float)
y_validation=y_validation.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)
y_test=y_test.astype(float)




#### Inizio Codice training

In [15]:
selector = SelectKBest(f_regression, k=5)
X_train_noName = selector.fit_transform(X_train_noName, y_train)
X_validation_noName = selector.transform(X_validation_noName)
X_test_noName = selector.transform(X_test_noName)

def evaluate_model(C, kernel, X_train, y_train, X_validation, y_validation):
    svm_model = SVC(C=C, kernel=kernel)
    svm_model = OneVsOneClassifier(svm_model)
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_validation)
    accuracy = accuracy_score(y_validation, y_pred)
    return accuracy

best_C = 1.0
best_kernel = 'linear'
best_accuracy = 0.0
max_iterations = 50

mask = ~np.isnan(y_validation)
y_validation = y_validation[mask]
X_validation_noName = X_validation_noName[mask] #Rimozione righe con NaN Values


for _ in range(max_iterations):
    new_C = random.uniform(0.1, 10.0)
    new_kernel = random.choice(['linear', 'rbf', 'poly', 'sigmoid'])
    accuracy = evaluate_model(new_C, new_kernel, X_train_noName, y_train, X_validation_noName, y_validation)
    if accuracy > best_accuracy:
        best_C = new_C
        best_kernel = new_kernel
        best_accuracy = accuracy

final_model = SVC(C=best_C, kernel=best_kernel)
final_model.fit(X_train_noName, y_train)
y_pred_validation = final_model.predict(X_validation_noName)
y_test_pred = final_model.predict(X_test_noName)

accuracy_test = accuracy_score(y_test, y_test_pred)
finalPrecision = precision_score(y_test, y_test_pred, average='weighted', zero_division=1)
finalRecall = recall_score(y_test, y_test_pred, average='weighted', zero_division=1)
print("Best Hyperparameters: C =", best_C, ", Kernel =", best_kernel)
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {finalPrecision}")
print(f"Recall: {finalRecall}")

Best Hyperparameters: C = 0.9343323567026282 , Kernel = rbf
Accuracy: 0.4921052631578947
Precision: 0.6090720221606648
Recall: 0.4921052631578947
